## 해설가 AI (KoGPT2 파인튜닝)

In [ ]:
# !pip install transformers[torch] datasets accelerate

^C


In [6]:
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    pipeline
)

### 2. 데이터셋 로드 및 전처리
만들어둔 `commentary_dataset.txt` 파일을 읽고, 파인튜닝에 사용할 수 있는 형태로 가공합니다.

In [ ]:
file_path = './data/commentary_dataset.txt'
data_pairs = []

with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

if text:
    blocks = text.split('---')
    for block in blocks:
        if 1 > len(block):
            continue
        _, input_line, output_line, _ = block.split('\n')
        input_text = input_line.replace('입력: ', '')
        output_text = output_line.replace('출력: ', '')
        data_pairs.append({'input': input_text, 'output': output_text})

print(f"총 {len(data_pairs)}개의 데이터 쌍을 로드했습니다.")
print(data_pairs[:2])

총 267개의 데이터 쌍을 로드했습니다.
[{'input': '아침에 늦잠을 잤다.', 'output': '아, 경기 시작부터 위기입니다! 중요한 순간, 늦잠이라는 치명적인 실수가 나오고 맙니다!'}, {'input': '코딩하는데 에러가 계속 난다.', 'output': '자, 키보드 위에서 손가락은 보이지 않고요! 엄청난 속도로 코드를 수정하고 있습니다만, 아~ 계속되는 에러! 쉽게 풀리지 않는 경기입니다!'}]


### 3. 모델 및 토크나이저 로드
`skt/KoGPT2-base-v2` 모델과 토크나이저를 불러오기

In [ ]:
model_name = 'skt/KoGPT2-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_name,
  bos_token='<|startoftext|>',
  eos_token='<|endoftext|>',
  pad_token='<|pad|>'
)

model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

config.json: 0.00B [00:00, ?B/s]

c:\Users\Playdata\AppData\Local\anaconda3\envs\nlp_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--skt--KoGPT2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(51203, 768)

### 4. 데이터셋 클래스 정의
KoGPT2 모델 학습을 위해 데이터를 특정 형식(`시작토큰` + `입력` + `출력` + `종료토큰`)으로 만들고 토크나이징하는 클래스를 정의합니다.

In [60]:
class CommentaryDataset(Dataset):
    def __init__(self, tokenizer, data_pairs, max_len=128):
        self.tokenizer = tokenizer
        self.data = []
        
        for pair in data_pairs:
            # 학습 형식: <|startoftext|>입력: [일상 문장] 출력: [해설 문장]<|endoftext|>
            formatted_text = (
                f"<|startoftext|>입력: {pair['input']} 출력: {pair['output']}<|endoftext|>"
            )
            
            # 텍스트를 토큰 ID로 변환
            tokenized = self.tokenizer(formatted_text,
                                       padding='max_length',
                                       truncation=True,
                                       max_length=max_len,
                                       return_tensors='pt')
            
            self.data.append({
                'input_ids': tokenized.input_ids.squeeze(0),
                'attention_mask': tokenized.attention_mask.squeeze(0),
                # Causal LM에서는 레이블이 입력과 동일합니다.
                'labels': tokenized.input_ids.squeeze(0).clone() 
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = CommentaryDataset(tokenizer, data_pairs)

### 5. 학습 설정 및 트레이너 생성
파인튜닝을 위한 여러 하이퍼파라미터를 설정하고, 학습을 담당할 `Trainer`를 생성합니다.

In [ ]:
output_dir = './results'

training_args = TrainingArguments(
    output_dir=output_dir,                # 결과물 저장 경로
    num_train_epochs=5,                   # 총 학습 에포크
    per_device_train_batch_size=4,        # 장치당 배치 사이즈
    warmup_steps=10,                      # 학습률 스케줄러를 위한 웜업 스텝
    weight_decay=0.01,                    # 가중치 감소
    logging_dir='./logs',                 # 로그 저장 경로
    logging_steps=10     
)

# 언어 모델링을 위한 데이터 콜레이터. MLM은 False로 설정합니다.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

### 6. 노트북으로도 할 수 있는 가벼운 파인튜닝 시작
이제 `trainer.train()`을 호출하여 실제 학습을 시작합니다.

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

### 7. 학습된 모델 저장
파인튜닝이 완료된 모델 로컬저장

In [2]:
save_dir = './fine-tuned-kogpt2-commentary'
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

NameError: name 'model' is not defined

### 8. 나만의 해설가 AI 테스트
저장된 모델을 `pipeline`으로 불러와 새로운 문장을 스포츠 해설처럼 생성하는지 테스트해봅니다.

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

In [2]:
save_dir = './fine-tuned-kogpt2-commentary'
tokenizer = AutoTokenizer.from_pretrained(save_dir,
  bos_token='<|startoftext|>',
  eos_token='<|endoftext|>',
  pad_token='<|pad|>'
)

model = AutoModelForCausalLM.from_pretrained(save_dir)

model.resize_token_embeddings(len(tokenizer))

Embedding(51203, 768)

In [3]:
tokenizer.save_pretrained(save_dir)
generator = pipeline('text-generation', model=save_dir, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

Device set to use cpu


In [6]:
while True:
    input_prompt = input("테스트할 문장을 입력하세요: (나가기=q)")
    if input_prompt == "q":
        break

    formatted_prompt = f"<|startoftext|>입력: {input_prompt} 출력:"

    print(f"입력 문장: {input_prompt}")

    # 텍스트 생성
    generated_output = generator(
        formatted_prompt, 
        max_length=100,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        top_k=100,
        top_p=0.3
    )

    print("AI 해설가 생성 문장: ")

    # 생성된 텍스트를 깔끔하게 정리합니다.
    full_text = generated_output[0]['generated_text']

    # '출력:' 이후의 텍스트만 잘라냅니다.
    start_index = full_text.find('출력:') + len('출력:')

    # 문장의 끝을 나타내는 eos_token 이후의 내용은 잘라냅니다.
    end_index = full_text.find(tokenizer.eos_token, start_index)
    if end_index == -1:
        end_index = len(full_text)

    commentary = full_text[start_index:end_index].strip()
    print(commentary)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


입력 문장: 공부를 한다.
AI 해설가 생성 문장: 
자신만의 루틴으로 승부하고 있습니다. 자신만의 플레이를 펼칩니다.


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


입력 문장: 공부를 하는데 어렵다. 춥다.
AI 해설가 생성 문장: 
어려운 상황에서도 굴하지 않는 강한 정신력을 가졌습니다! 위기 상황에서 더욱 빛나는 선수입니다.
